<h2>
    <p>
        MÔN BIG DATA 2019
    </p>
    <p style="color:red">
        Nhóm 13 - Le Nguyen Son Nguyen CH1702039
    </p>
</h2>

<h3>
    <p>
        SỬ DỤNG SPARK VÀ SPARK MACHINE LEARNING LIBRARY ĐỂ TRAIN MODEL PHÂN TÍCH NGỮ NGHĨA DÙNG MULTILAYER PERCEPTRONS (BASIC DEEP LEARNING)
    </p>
    <p style="color: blue">
        Phần áp dụng Model đã train vào phân tích dữ liệu Real Time sử dụng Kafka
    </p>
     <img src="https://i.ibb.co/dWWqFbN/Drawing4.jpg" alt="" height="100%" width="100%"> 
</h3>

<h4>
    <p style="color:blue">
        Khai báo các biến môi trường để Spark sử dụng trong quá trình chạy
    </p>
</h4>

In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/home/spark/spark-2.4.3-bin-hadoop2.7"
#os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local --packages org.apache.spark:spark-streaming-kafka-0-8:2.3.0 pyspark-shell"

In [2]:
import findspark
findspark.init()

<h4>
    <p style="color:red">
        Import các thư viện của PySpark
    </p>
</h4>

In [3]:
from os import listdir
from pyspark import SparkContext
from pyspark.ml import PipelineModel
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import *

<h4>
    <p style="color:blue">
        Khai báo số memory mà mỗi Worker sẽ sử dụng để thực thi task
    </p>
</h4>

In [4]:
SparkContext.setSystemProperty('spark.executor.memory', '6656m')

<h4>
    <p style="color:red">
        Khai báo địa chỉ của Spark master, lưu ý là phải thay đổi địa chỉ cho phù hợp với từng Spark cluster
    </p>
</h4>

In [5]:
# tạo spark context, lưu ý nhớ thay đổi lại địa chỉ của spark cho phù hợp
sc = SparkContext("spark://10.255.255.6:7077", "Kafka Semantic Handle Real Time")
spark = SparkSession.builder.getOrCreate()

<h4>
    <p style="color:red">
      Sử dụng lại model (Load Model) đã lưu sử dụng lệnh dưới đây
    </p>
</h4>

In [6]:
pl_model = PipelineModel.load("hdfs://s14e18f4e58324b66b78ecd8c831a0c6413-master.uitlab.com:9000/sparkdata/pl2.model")
pl_model

PipelineModel_16ec0b25fffe

<h4>
    <p style="color:blue">
        Import các thư viện của PySpark Streaming và Kafka sử dụng trong Spark
    </p>
</h4>

In [7]:
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from pyspark.sql.types import *

<h4>
    <p style="color:red">
        Khai báo địa chỉ của Kafka Cluster và topic để lấy dữ liệu
    </p>
</h4>

In [8]:
kafkaBroker = 'localhost:9092'
topic = 'spark-sematic-input1'
batchTimeDur = 1

<h4>
    <p style="color:blue">
        Tạo đối tượng Streaming, với Time là 1s sẽ fetch dữ liệu một lần
    </p>
</h4>

In [9]:
ssc = StreamingContext(sc, batchTimeDur)

<h4>
    <p style="color:red">
        Tạo luồng Streaming Kafka ứng với Spark Context và Topic đã khai báo
    </p>
</h4>

In [10]:
directKafkaStream = KafkaUtils.createDirectStream(ssc, [topic], {"metadata.broker.list": kafkaBroker})

<h4>
    <p>
        Import thư viện xử lý Kafka
    </p>
</h4>

In [11]:
from kafka import KafkaProducer
import json

<h4>
    <p style="color:blue">
        Tạo Kafka Producer để đẩy dữ liệu sau khi đã đi qua Model Deep Learning quay ngược trở về Kafka, cung cấp dữ liệu cho các nguồn khác.
    </p>
</h4>

In [12]:
producer = KafkaProducer(bootstrap_servers='localhost:9092')

<h4>
    <p style="color:red">
        Function để xử lý dữ liệu trong Streaming
    </p>
</h4>

In [13]:
def handler(message):
    records = message.collect()
    data = []
    index = 1
    for record in records:
        key, value = record
        #print(value)
        data.append((index, value))
        index = index + 1
    if len(data) > 0:
        #print(data)
        sentenceDataFrame = spark.createDataFrame(data, ["id", "sentence"])
        new_dt = pl_model.transform(sentenceDataFrame)
        new_dt = new_dt.select("sentence", "prediction").collect()
        
        for row in new_dt:
            s = row['sentence']
            p = row['prediction']
            
            o = {
                'sentence': s,
                'prediction': p
            }
            
            j = json.dumps(o)
            
            print(j)
            
            j = str.encode(j)
            
            producer.send("spark-sematic-output1", j)
            producer.flush()
            
            #print(j)
            
        
        #print(new_dt)

<h4>
    <p>
        Đăng ký funtion cho luồng Kafka Streaming
    </p>
</h4>

In [14]:
directKafkaStream.foreachRDD(handler)

<h4>
    <p style="color:blue">
        Start Streaming và xem kết quả.
    </p>
    <p style="color:red">
        Mỗi khi có luồng dữ liệu đến, data sẽ chạy qua model để được giá trị prediction.
    </p>
    <p style="color:red">
        Sau khi có kết quả, data kết quả sẽ được submit lên một topic Kafka mới, Topic output này sẽ là nguồn dữ liệu cho việc lưu trữ xuống MongoDB hoặc được hiển thị lên biểu đồ dạng Real Time.
    </p>
</h4>

In [ ]:
ssc.start()
ssc.awaitTermination()

{"prediction": 1.0, "sentence": "I only went to see this movie because I have always liked Kevin Costner. I felt that Ashton did a great job in the Butterfly Effect. Unfortunately, even though these two actors were/are capable of good if not great acting moments some of that was missing here. Some of the scenes were just not believable and didn't have enough story line support.<br /><br />Though the movie claims influence from the hurricane Katrina aftermath, there was very little (none) to that effect in the movie.<br /><br />Overall, I liked the fact that the movie brought forward some of what goes into saving lives from a water perspective.<br /><br />The special effects were pretty good and more than a little intimidating. Not sure I'll ever go deep sea fishing again...<br /><br />I expected a little more emotion in the film than what was presented.<br /><br />Definitely a movie that could've been seen on DVD."}
{"prediction": 0.0, "sentence": "\"The Dresser\" is perhaps the most r

<h4>
    <p>
        Kết Thúc một phiên làm việc trong Spark
    </p>
</h4>

In [14]:
print("end_of_code")
spark.stop()

end_of_code
